In [3]:
import numpy as np
import random
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

In [122]:
hidden_size = 200   # 根据实际情况决定
num_moves = 2811   # =动作数量=文件数量
key_points = ['nose','leftEye','rightEye','leftEar','rightEar','leftShoulder','rightShoulder','leftElbow','rightElbow','leftWrist','rightWrist','leftHip','rightHip','leftKnee','rightKnee','leftAnkle','rightAnkle']

class JointsDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.moves = []
        for i in range(num_moves):
            with open('./train_data/' + 'train_'+ str(i) + '.json') as f:  # 根据具体文件名格式改
                l = json.load(f)
                self.moves.append(l)
        
        # for i in range(num_moves):
        #     with open('./train_data/' + str(i) + '.txt') as f:
        #        content = f.read()
        #        frames = content.splitlines()
        #        self.moves.append(frames)
        
    
    def __getitem__(self, index):
        move = self.moves[index]
        y1 = torch.empty(1, dtype=torch.long)  #label
        y1[0] = int(move[0])
        frams_list = []
        for frame in move[1:]:  # 处理json文件中的list和dict，因为没有data就还没测试过，可能有错
            if frame == {}:
                continue
            joints = []
            for key in key_points:
                if (str(frame[key]) == '0'):
                    x = y = 0
                else:
                    coordinates = str(frame[key]).split(',')
                    x,y = float(coordinates[0]),float(coordinates[1])
                joints.append(x)
                joints.append(y)
            frams_list.append(joints)
        x = torch.Tensor(frams_list)
        return x, y1
    
    def __len__(self):
        return len(self.moves)

In [112]:

class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        # TO DO
        # YOUR CODE HERE
        
        self.in_size = 34   # =一帧坐标的关键点数量
        self.hidden_size = 100   #随便取，比in size大一截就行，因为posenet只有十几个点，最好改成100左右
        self.out_size = 3   # label数量
        self.lstm = nn.LSTM(self.in_size, self.hidden_size)
        self.h2o = nn.Linear(self.hidden_size,self.out_size)
       
    
    def forward(self,x):
        # TO DO
        # YOUR CODE HERE
        lstm_out, _ = self.lstm(x)
        out_space = self.h2o(lstm_out)
        #y_pred = F.softmax(out_space)
        return out_space
        
        
        # x = x.view(self.in_size)
        # waa_a = self.h2h(a)
        # wax_x = self.i2h(x)
        # new_a = torch.tanh(waa_a + wax_x)
        # y = self.h2o(new_a)
        # return new_a

In [106]:
# utility for getting prediction accuracy
def get_correct(y_pred, y):
    # find the prediction class label
    _ ,pred_class = y_pred.max(dim=1)
    return pred_class == y

In [126]:
import random

trn_ds = JointsDataset()
trn_dl = DataLoader(trn_ds, batch_size=1, shuffle=True)

def train(trn_ds, trn_dl, epochs=1):
    # Create a new model, loss_fn and optimizer.
    model = RNN()
    # Use cross entropy loss
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(),lr=0.0001) # lr: learning rate，学习速度
    h_prev = torch.zeros((1,hidden_size))
    max_train_accu = 0
    max_vali_accu = 0
    
    train = []
    vali = []
    for move_num, (x,y) in enumerate(trn_dl):   # 取80%为training set, 20%为validation set
        dice = random.random()
        if dice <= 0.8:
            train.append([x,y])
        else:
            vali.append([x,y])
        
        
    for e in range(1, epochs+1):
        print(f'{"-"*20} Epoch {e} {"-"*20}')
        #count = 0
        train_count = num_moves*0.8
        vali_count = num_moves*0.2
        train_num_correct = 0
        vali_num_correct = 0
        train_loss = 0
        vali_loss = 0
        
        for (x,y) in train:
            #if count <= train_count:
                #count += 1
                #print('----',count,'----')
                y = y.view(y.shape[1])
                model.train()
                
                optimizer.zero_grad()
              
                y_pred = model.forward(x)[0][-1]
                y_pred = y_pred.view(1,len(y_pred))
                label = y.view(1)
                #print('y_pred', y_pred)
                #print('label',label)
                l = loss_fn(y_pred, label)
                train_loss += l   
                correct = get_correct(y_pred,label)
                if correct == True:
                    train_num_correct += 1
                #print('train num of correct',train_num_correct)
                l.backward(retain_graph=True)
        
                # Clip your gradients
                torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        
                # Update parameters
                optimizer.step()
                # print('loss',loss)
        for (x,y) in vali:
                #print('----',count,'----')
                #count+=1
                y = y.view(y.shape[1])
                y_pred = model.forward(x)[0][-1]
                y_pred = y_pred.view(1,len(y_pred))
                label = y.view(1)
                #print('y_pred', y_pred)
                #print('label',label)
                l = loss_fn(y_pred, label)
                vali_loss += l   
                correct = get_correct(y_pred,label)
                if correct == True:
                    vali_num_correct += 1
                #print('vali num of correct',vali_num_correct)
        
        train_accuracy = train_num_correct/len(train)
        vali_accuracy = vali_num_correct/len(vali)
        print('TRAIN ACCURACY',train_accuracy,'VALIDATION ACCURACY', vali_accuracy)
        #if train_accuracy > max_train_accu and vali_accuracy > max_vali_accu:
        #      torch.save(model.state_dict(),PATH)
        PATH = 'data0829MAX '+str(e)+' .pth'
        torch.save(model.state_dict(),PATH)   
        

In [ ]:
#Start training
train(trn_ds, trn_dl, epochs=200)   